In [37]:
import pandas as pd
import geopandas as gpd
from shapely import wkt
import folium

In [38]:
df = pd.read_csv("processed/selected_cities_20241117.csv")
full = pd.read_csv("processed/full_calculation_20241117.csv")

In [39]:
# Convert the 'geometry' column from WKT to shapely geometries
df['geometry'] = df['geometry'].apply(wkt.loads)

In [40]:
gdf = gpd.GeoDataFrame(df, geometry='geometry', crs='EPSG:3857')
gdf = gdf.to_crs(epsg=4326) # Transform to WGS84 (EPSG:4326)

In [41]:
# WGS84 (EPSG:4326)
sgdf = gdf[["FID","CITY_NAME","CNTRY_NAME","FIPS_CNTRY","geometry","POP_updated","POP_rank_updated","capital","distance_sea","city_type","pop_diff"]]

In [42]:
# 229 countries in total (selected cities)
sgdf['FIPS_CNTRY'].value_counts()

FIPS_CNTRY
SL    2
LY    2
PP    2
GB    2
GA    2
     ..
JO    1
KG    1
KR    1
KT    1
ZI    1
Name: count, Length: 229, dtype: int64

In [43]:
# 229 capital cities
sgdf[sgdf.capital==1]

,FID,CITY_NAME,CNTRY_NAME,FIPS_CNTRY,geometry,POP_updated,POP_rank_updated,capital,distance_sea,city_type,pop_diff
0,296,Oranjestad,Aruba,AA,POINT (-70.02600 12.52500),81915,6,1,1.449000,coast,NaN
1,1556,Saint John's,Antigua & Barbuda,AC,POINT (-61.84218 17.12478),185565,5,1,0.684450,coast,NaN
2,2257,Abu Dhabi,United Arab Emirates,AE,POINT (54.37100 24.47600),1202756,2,1,2.347458,coast,NaN
3,1072,Kabul,Afghanistan,AF,POINT (69.13676 34.53091),4434600,2,1,1192.107975,inland,NaN
4,1910,Algiers,Algeria,AG,POINT (3.05000 36.78330),2364230,2,1,1.553963,coast,NaN
...,...,...,...,...,...,...,...,...,...,...,...
320,17,Apia,Samoa,WS,POINT (-171.73949 -13.83702),16585,7,1,0.000000,coast,NaN
321,2458,Mbabane,Eswatini,WZ,POINT (31.19130 -26.30338),60691,6,1,157.163254,inland,NaN
322,865,Sanaa,Yemen,YM,POINT (44.20950 15.36144),33908,7,1,149.616375,inland,NaN
324,2382,Lusaka,Zambia,ZA,POINT (28.17000 -15.43000),207350,5,1,897.500478,inland,NaN


In [44]:
# number of non capital cities
326-229

97

In [45]:
# countries that only have one city (selected cities)
country_city_counts = sgdf['FIPS_CNTRY'].value_counts()
fips_132 = sgdf[sgdf['FIPS_CNTRY'].isin(country_city_counts[country_city_counts==1].index)].FIPS_CNTRY

In [46]:
# countries that only have one city (selected cities)
sgdf[sgdf['FIPS_CNTRY'].isin(country_city_counts[country_city_counts==1].index)]

,FID,CITY_NAME,CNTRY_NAME,FIPS_CNTRY,geometry,POP_updated,POP_rank_updated,capital,distance_sea,city_type,pop_diff
0,296,Oranjestad,Aruba,AA,POINT (-70.02600 12.52500),81915,6,1,1.449000,coast,NaN
1,1556,Saint John's,Antigua & Barbuda,AC,POINT (-61.84218 17.12478),185565,5,1,0.684450,coast,NaN
2,2257,Abu Dhabi,United Arab Emirates,AE,POINT (54.37100 24.47600),1202756,2,1,2.347458,coast,NaN
3,1072,Kabul,Afghanistan,AF,POINT (69.13676 34.53091),4434600,2,1,1192.107975,inland,NaN
6,2260,Baku,Azerbaijan,AJ,POINT (49.81600 40.32400),1291000,2,1,1364.705622,inland,NaN
...,...,...,...,...,...,...,...,...,...,...,...
319,16,Mata-Utu,Wallis & Futuna,WF,POINT (-176.15156 -13.27330),508200,3,1,0.018889,coast,NaN
320,17,Apia,Samoa,WS,POINT (-171.73949 -13.83702),16585,7,1,0.000000,coast,NaN
321,2458,Mbabane,Eswatini,WZ,POINT (31.19130 -26.30338),60691,6,1,157.163254,inland,NaN
324,2382,Lusaka,Zambia,ZA,POINT (28.17000 -15.43000),207350,5,1,897.500478,inland,NaN


In [47]:
(229-132)*2+132

326

In [48]:
# countries that only have one city (original dataset) -- including 2 countries without capitals
counts = full['FIPS_CNTRY'].value_counts()
fips_82 = full[full['FIPS_CNTRY'].isin(counts[counts==1].index)].FIPS_CNTRY

In [49]:
full[full['FIPS_CNTRY'].isin(counts[counts==1].index)]

,FID,GMI_ADMIN,ADMIN_NAME,FIPS_CNTRY,CNTRY_NAME,STATUS,POP,POP_RANK,POP_CLASS,POP_SOURCE,...,Name1__sea,Name2__sea,Name3__sea,TYPE__sea,ISO_CC__sea,SHAPE_Length__sea,SHAPE_Area__sea,distance_sea,POP_rank_updated,city_type
15,16,WLF,Wallis & Futuna,WF,Wallis & Futuna,National and provincial capital,0,0,,,...,Pacific Ocean,NaN,NaN,Ocean or Sea,NaN,1.652176,0.055174,0.018889,3,coast
16,17,WSM,Samoa,WS,Samoa,National capital,37391,7,"Less than 50,000",UN_Data_2010_2020,...,Pacific Ocean,NaN,NaN,Ocean or Sea,NaN,1.146609,0.045098,0.000000,7,coast
17,18,ASM,American Samoa,AQ,American Samoa,National and provincial capital,3656,7,"Less than 50,000",UN_Data_2010_2020,...,Pacific Ocean,NaN,NaN,Ocean or Sea,NaN,2.595805,0.050280,0.302533,7,coast
18,19,PYF,French Polynesia,FP,French Polynesia,National and provincial capital,25763,7,"Less than 50,000",UN_Data_2010_2020,...,Pacific Ocean,NaN,NaN,Ocean or Sea,NaN,1.182310,0.049963,0.052122,7,coast
19,20,NIU,Niue,NE,Niue,National and provincial capital,639,7,"Less than 50,000",UN_Data_2010_2020,...,Pacific Ocean,NaN,NaN,Ocean or Sea,NaN,1.181183,0.043316,0.826935,7,coast
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2523,2524,SLB,Solomon Is.,BP,Solomon Is.,National capital,0,0,,,...,Solomon Sea,Pacific Ocean,NaN,Ocean or Sea,NaN,0.506198,0.011142,0.583022,4,coast
2524,2525,VUT,Vanuatu,NH,Vanuatu,National capital,20094,7,"Less than 50,000",UN_Data_2010_2020,...,Pacific Ocean,NaN,NaN,Ocean or Sea,NaN,0.043550,0.000072,0.536871,4,coast
2525,2526,FJI,Fiji,FJ,Fiji,National capital,0,0,,,...,Pacific Ocean,NaN,NaN,Ocean or Sea,NaN,1.733989,0.050104,1.375259,5,coast
2526,2527,NCL,New Caledonia,NC,New Caledonia,National and provincial capital,99926,6,"50,000 to 99,999",UN_Data_2010_2020,...,Coral Sea,Pacific Ocean,NaN,Ocean or Sea,NaN,0.720061,0.008137,13.069313,6,inland


In [50]:
len(sgdf.FIPS_CNTRY.unique())

229

In [51]:
len(full.FIPS_CNTRY.unique()) 

231

In [52]:
# two countries in the original dataset do not have capital cities
set(full.FIPS_CNTRY.unique()) - set(sgdf.FIPS_CNTRY.unique())

{'GZ', 'WE'}

In [53]:
full[full.FIPS_CNTRY=="GZ"]
#full[full.FIPS_CNTRY=="WE"]

,FID,GMI_ADMIN,ADMIN_NAME,FIPS_CNTRY,CNTRY_NAME,STATUS,POP,POP_RANK,POP_CLASS,POP_SOURCE,...,Name1__sea,Name2__sea,Name3__sea,TYPE__sea,ISO_CC__sea,SHAPE_Length__sea,SHAPE_Area__sea,distance_sea,POP_rank_updated,city_type
812,813,ISR-GAZ,Gaza Strip,GZ,Gaza Strip,Other,0,0,,,...,Mediterranean Sea,Atlantic Ocean,NaN,Ocean or Sea,NaN,1.011,0.058621,1.174571,3,coast


In [54]:
# one-city countries due to selection
len(set(fips_132) - set(fips_82)) 

52

### Only one city in a country: 80 one-city countries in the original dataset. 52 countries due to selection.

In [19]:
# the 52 countries
fips_52 = fips_132[~fips_132.isin(fips_82)]
fips_52

2      AE
3      AF
6      AJ
18     AU
20     BA
22     BC
24     BE
31     BL
36     BO
40     BT
41     BU
43     BY
46     CB
47     CD
52     CG
67     CT
93     ET
94     EZ
109    GG
134    HU
149    IZ
155    JO
158    KG
167    KZ
168    LA
175    LO
177    LT
178    LU
186    MG
188    MI
191    ML
199    MU
209    NG
217    NP
226    OD
227    PA
248    RO
249    RP
250    RQ
253    RW
273    ST
281    SZ
285    TI
290    TP
297    TX
300    UG
303    UP
306    UV
309    UZ
321    WZ
324    ZA
325    ZI
Name: FIPS_CNTRY, dtype: object

In [33]:
# the 52 countries (one city selected but have 2+ cities in the original dataset)
cntry52 = full[full['FIPS_CNTRY'].isin(fips_52)]
cntry52

,FID,GMI_ADMIN,ADMIN_NAME,FIPS_CNTRY,CNTRY_NAME,STATUS,POP,POP_RANK,POP_CLASS,POP_SOURCE,...,Name1__sea,Name2__sea,Name3__sea,TYPE__sea,ISO_CC__sea,SHAPE_Length__sea,SHAPE_Area__sea,distance_sea,POP_rank_updated,city_type
4,5,PRY-AMM,Amambay,PA,Paraguay,Provincial capital,0,0,,,...,Atlantic Ocean,NaN,NaN,Ocean or Sea,NaN,1.064543,0.007229,852.927751,5,inland
5,6,PRY-CNN,Canindeyu,PA,Paraguay,Provincial capital,0,0,,,...,Atlantic Ocean,NaN,NaN,Ocean or Sea,NaN,1.064543,0.007229,637.647981,7,inland
6,7,PRY-APA,Alto Parana,PA,Paraguay,Provincial capital,0,0,,,...,Atlantic Ocean,NaN,NaN,Ocean or Sea,NaN,0.355920,0.000982,650.833576,4,inland
7,8,PRY-ITA,Itapua,PA,Paraguay,Provincial capital,0,0,,,...,Atlantic Ocean,NaN,NaN,Ocean or Sea,NaN,0.591384,0.019002,716.754502,5,inland
14,15,BOL-LPA,La Paz,BL,Bolivia,National and provincial capital,2143000,2,"1,000,000 to 4,999,999",UN_Habitat_2020,...,Pacific Ocean,NaN,NaN,Ocean or Sea,NaN,0.969157,0.051438,329.256259,2,inland
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2479,2480,LSO-QTH,Quthing,LT,Lesotho,Provincial capital,0,0,,,...,Indian Ocean,NaN,NaN,Ocean or Sea,NaN,0.039226,0.000028,251.620148,7,inland
2484,2485,LSO-BBU,Butha-Buthe,LT,Lesotho,Provincial capital,0,0,,,...,Indian Ocean,NaN,NaN,Ocean or Sea,NaN,0.049334,0.000036,336.963531,7,inland
2485,2486,LSO-MKH,Mokhotlong,LT,Lesotho,Provincial capital,0,0,,,...,Indian Ocean,NaN,NaN,Ocean or Sea,NaN,0.049334,0.000036,222.236571,7,inland
2487,2488,LSO-QNE,Qacha's Nek,LT,Lesotho,Provincial capital,0,0,,,...,Indian Ocean,NaN,NaN,Ocean or Sea,NaN,0.038749,0.000024,206.518650,7,inland


In [34]:
#cntry52.to_csv("processed/cntry52_20241118.csv", index=False) #WGS84 (EPSG:4326)

### Batches

In [55]:
sgdf.head(10)

,FID,CITY_NAME,CNTRY_NAME,FIPS_CNTRY,geometry,POP_updated,POP_rank_updated,capital,distance_sea,city_type,pop_diff
0,296,Oranjestad,Aruba,AA,POINT (-70.02600 12.52500),81915,6,1,1.449000,coast,NaN
1,1556,Saint John's,Antigua & Barbuda,AC,POINT (-61.84218 17.12478),185565,5,1,0.684450,coast,NaN
2,2257,Abu Dhabi,United Arab Emirates,AE,POINT (54.37100 24.47600),1202756,2,1,2.347458,coast,NaN
3,1072,Kabul,Afghanistan,AF,POINT (69.13676 34.53091),4434600,2,1,1192.107975,inland,NaN
4,1910,Algiers,Algeria,AG,POINT (3.05000 36.78330),2364230,2,1,1.553963,coast,NaN
5,1927,Batna,Algeria,AG,POINT (6.17200 35.55700),1119791,2,0,171.612108,inland,1244439.0
6,2260,Baku,Azerbaijan,AJ,POINT (49.81600 40.32400),1291000,2,1,1364.705622,inland,NaN
7,631,Tirana,Albania,AL,POINT (19.83200 41.33200),418495,4,1,36.166383,inland,NaN
8,605,Durres,Albania,AL,POINT (19.45400 41.32700),113249,5,0,1.825464,coast,305246.0
9,781,Yerevan,Armenia,AM,POINT (44.53268 40.20803),1086677,2,1,1047.205463,inland,NaN


In [56]:
# first_batch = two cities selected in each country
sgdf['first_batch'] = sgdf.groupby('FIPS_CNTRY')['FIPS_CNTRY'].transform('count').apply(lambda x: 1 if x > 1 else 0)

/Users/mengcai/opt/anaconda3/lib/python3.11/site-packages/geopandas/geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [58]:
first = sgdf[sgdf.first_batch==1]

In [60]:
#first.to_csv("processed/first20241118.csv", index=False) # WGS84 (EPSG:4326)

In [62]:
# FIPS of the second batch
sgdf[sgdf.first_batch==0].FIPS_CNTRY 

0      AA
1      AC
2      AE
3      AF
6      AJ
       ..
319    WF
320    WS
321    WZ
324    ZA
325    ZI
Name: FIPS_CNTRY, Length: 132, dtype: object

In [67]:
# pool of the second batch of cities
pool = full[full['FIPS_CNTRY'].isin(sgdf[sgdf.first_batch==0].FIPS_CNTRY)]
pool

,FID,GMI_ADMIN,ADMIN_NAME,FIPS_CNTRY,CNTRY_NAME,STATUS,POP,POP_RANK,POP_CLASS,POP_SOURCE,...,Name1__sea,Name2__sea,Name3__sea,TYPE__sea,ISO_CC__sea,SHAPE_Length__sea,SHAPE_Area__sea,distance_sea,POP_rank_updated,city_type
4,5,PRY-AMM,Amambay,PA,Paraguay,Provincial capital,0,0,,,...,Atlantic Ocean,NaN,NaN,Ocean or Sea,NaN,1.064543,0.007229,852.927751,5,inland
5,6,PRY-CNN,Canindeyu,PA,Paraguay,Provincial capital,0,0,,,...,Atlantic Ocean,NaN,NaN,Ocean or Sea,NaN,1.064543,0.007229,637.647981,7,inland
6,7,PRY-APA,Alto Parana,PA,Paraguay,Provincial capital,0,0,,,...,Atlantic Ocean,NaN,NaN,Ocean or Sea,NaN,0.355920,0.000982,650.833576,4,inland
7,8,PRY-ITA,Itapua,PA,Paraguay,Provincial capital,0,0,,,...,Atlantic Ocean,NaN,NaN,Ocean or Sea,NaN,0.591384,0.019002,716.754502,5,inland
14,15,BOL-LPA,La Paz,BL,Bolivia,National and provincial capital,2143000,2,"1,000,000 to 4,999,999",UN_Habitat_2020,...,Pacific Ocean,NaN,NaN,Ocean or Sea,NaN,0.969157,0.051438,329.256259,2,inland
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2523,2524,SLB,Solomon Is.,BP,Solomon Is.,National capital,0,0,,,...,Solomon Sea,Pacific Ocean,NaN,Ocean or Sea,NaN,0.506198,0.011142,0.583022,4,coast
2524,2525,VUT,Vanuatu,NH,Vanuatu,National capital,20094,7,"Less than 50,000",UN_Data_2010_2020,...,Pacific Ocean,NaN,NaN,Ocean or Sea,NaN,0.043550,0.000072,0.536871,4,coast
2525,2526,FJI,Fiji,FJ,Fiji,National capital,0,0,,,...,Pacific Ocean,NaN,NaN,Ocean or Sea,NaN,1.733989,0.050104,1.375259,5,coast
2526,2527,NCL,New Caledonia,NC,New Caledonia,National and provincial capital,99926,6,"50,000 to 99,999",UN_Data_2010_2020,...,Coral Sea,Pacific Ocean,NaN,Ocean or Sea,NaN,0.720061,0.008137,13.069313,6,inland


In [68]:
# if city_type of all rows with the same  FIPS_CNTRY is "inland",  give "all_inland" value 1, otherwise 0
pool['all_inland'] = pool.groupby('FIPS_CNTRY')['city_type'].transform(
    lambda x: 1 if all(val == "inland" for val in x) else 0
)

/var/folders/1v/222nq_l91hg0lcmf2d4rfps80000gn/T/ipykernel_29684/3819384403.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pool['all_inland'] = pool.groupby('FIPS_CNTRY')['city_type'].transform(


In [69]:
pool

,FID,GMI_ADMIN,ADMIN_NAME,FIPS_CNTRY,CNTRY_NAME,STATUS,POP,POP_RANK,POP_CLASS,POP_SOURCE,...,Name2__sea,Name3__sea,TYPE__sea,ISO_CC__sea,SHAPE_Length__sea,SHAPE_Area__sea,distance_sea,POP_rank_updated,city_type,all_inland
4,5,PRY-AMM,Amambay,PA,Paraguay,Provincial capital,0,0,,,...,NaN,NaN,Ocean or Sea,NaN,1.064543,0.007229,852.927751,5,inland,1
5,6,PRY-CNN,Canindeyu,PA,Paraguay,Provincial capital,0,0,,,...,NaN,NaN,Ocean or Sea,NaN,1.064543,0.007229,637.647981,7,inland,1
6,7,PRY-APA,Alto Parana,PA,Paraguay,Provincial capital,0,0,,,...,NaN,NaN,Ocean or Sea,NaN,0.355920,0.000982,650.833576,4,inland,1
7,8,PRY-ITA,Itapua,PA,Paraguay,Provincial capital,0,0,,,...,NaN,NaN,Ocean or Sea,NaN,0.591384,0.019002,716.754502,5,inland,1
14,15,BOL-LPA,La Paz,BL,Bolivia,National and provincial capital,2143000,2,"1,000,000 to 4,999,999",UN_Habitat_2020,...,NaN,NaN,Ocean or Sea,NaN,0.969157,0.051438,329.256259,2,inland,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2523,2524,SLB,Solomon Is.,BP,Solomon Is.,National capital,0,0,,,...,Pacific Ocean,NaN,Ocean or Sea,NaN,0.506198,0.011142,0.583022,4,coast,0
2524,2525,VUT,Vanuatu,NH,Vanuatu,National capital,20094,7,"Less than 50,000",UN_Data_2010_2020,...,NaN,NaN,Ocean or Sea,NaN,0.043550,0.000072,0.536871,4,coast,0
2525,2526,FJI,Fiji,FJ,Fiji,National capital,0,0,,,...,NaN,NaN,Ocean or Sea,NaN,1.733989,0.050104,1.375259,5,coast,0
2526,2527,NCL,New Caledonia,NC,New Caledonia,National and provincial capital,99926,6,"50,000 to 99,999",UN_Data_2010_2020,...,Pacific Ocean,NaN,Ocean or Sea,NaN,0.720061,0.008137,13.069313,6,inland,1


In [75]:
pool[pool.all_inland==1]

,FID,GMI_ADMIN,ADMIN_NAME,FIPS_CNTRY,CNTRY_NAME,STATUS,POP,POP_RANK,POP_CLASS,POP_SOURCE,...,Name3__sea,TYPE__sea,ISO_CC__sea,SHAPE_Length__sea,SHAPE_Area__sea,distance_sea,POP_rank_updated,city_type,all_inland,all_coast
4,5,PRY-AMM,Amambay,PA,Paraguay,Provincial capital,0,0,,,...,NaN,Ocean or Sea,NaN,1.064543,0.007229,852.927751,5,inland,1,0
5,6,PRY-CNN,Canindeyu,PA,Paraguay,Provincial capital,0,0,,,...,NaN,Ocean or Sea,NaN,1.064543,0.007229,637.647981,7,inland,1,0
6,7,PRY-APA,Alto Parana,PA,Paraguay,Provincial capital,0,0,,,...,NaN,Ocean or Sea,NaN,0.355920,0.000982,650.833576,4,inland,1,0
7,8,PRY-ITA,Itapua,PA,Paraguay,Provincial capital,0,0,,,...,NaN,Ocean or Sea,NaN,0.591384,0.019002,716.754502,5,inland,1,0
14,15,BOL-LPA,La Paz,BL,Bolivia,National and provincial capital,2143000,2,"1,000,000 to 4,999,999",UN_Habitat_2020,...,NaN,Ocean or Sea,NaN,0.969157,0.051438,329.256259,2,inland,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2484,2485,LSO-BBU,Butha-Buthe,LT,Lesotho,Provincial capital,0,0,,,...,NaN,Ocean or Sea,NaN,0.049334,0.000036,336.963531,7,inland,1,0
2485,2486,LSO-MKH,Mokhotlong,LT,Lesotho,Provincial capital,0,0,,,...,NaN,Ocean or Sea,NaN,0.049334,0.000036,222.236571,7,inland,1,0
2487,2488,LSO-QNE,Qacha's Nek,LT,Lesotho,Provincial capital,0,0,,,...,NaN,Ocean or Sea,NaN,0.038749,0.000024,206.518650,7,inland,1,0
2498,2499,,Central Equatoria,OD,South Sudan,National and provincial capital,0,0,,,...,NaN,Ocean or Sea,NaN,0.604022,0.002691,1274.995948,5,inland,1,0


In [72]:
# if city_type of all rows with the same  FIPS_CNTRY is "coast",  give "all_coast" value 1, otherwise 0
pool['all_coast'] = pool.groupby('FIPS_CNTRY')['city_type'].transform(
    lambda x: 1 if all(val == "coast" for val in x) else 0
)

/var/folders/1v/222nq_l91hg0lcmf2d4rfps80000gn/T/ipykernel_29684/4116961514.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pool['all_coast'] = pool.groupby('FIPS_CNTRY')['city_type'].transform(


In [77]:
pool[pool.all_coast==1]

,FID,GMI_ADMIN,ADMIN_NAME,FIPS_CNTRY,CNTRY_NAME,STATUS,POP,POP_RANK,POP_CLASS,POP_SOURCE,...,Name3__sea,TYPE__sea,ISO_CC__sea,SHAPE_Length__sea,SHAPE_Area__sea,distance_sea,POP_rank_updated,city_type,all_inland,all_coast
15,16,WLF,Wallis & Futuna,WF,Wallis & Futuna,National and provincial capital,0,0,,,...,NaN,Ocean or Sea,NaN,1.652176,0.055174,0.018889,3,coast,0,1
16,17,WSM,Samoa,WS,Samoa,National capital,37391,7,"Less than 50,000",UN_Data_2010_2020,...,NaN,Ocean or Sea,NaN,1.146609,0.045098,0.000000,7,coast,0,1
17,18,ASM,American Samoa,AQ,American Samoa,National and provincial capital,3656,7,"Less than 50,000",UN_Data_2010_2020,...,NaN,Ocean or Sea,NaN,2.595805,0.050280,0.302533,7,coast,0,1
18,19,PYF,French Polynesia,FP,French Polynesia,National and provincial capital,25763,7,"Less than 50,000",UN_Data_2010_2020,...,NaN,Ocean or Sea,NaN,1.182310,0.049963,0.052122,7,coast,0,1
19,20,NIU,Niue,NE,Niue,National and provincial capital,639,7,"Less than 50,000",UN_Data_2010_2020,...,NaN,Ocean or Sea,NaN,1.181183,0.043316,0.826935,7,coast,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2522,2523,TUV,Tuvalu,TV,Tuvalu,National capital,6152,7,"Less than 50,000",UN_Data_2010_2020,...,NaN,Ocean or Sea,NaN,1.486024,0.061388,0.000000,7,coast,0,1
2523,2524,SLB,Solomon Is.,BP,Solomon Is.,National capital,0,0,,,...,NaN,Ocean or Sea,NaN,0.506198,0.011142,0.583022,4,coast,0,1
2524,2525,VUT,Vanuatu,NH,Vanuatu,National capital,20094,7,"Less than 50,000",UN_Data_2010_2020,...,NaN,Ocean or Sea,NaN,0.043550,0.000072,0.536871,4,coast,0,1
2525,2526,FJI,Fiji,FJ,Fiji,National capital,0,0,,,...,NaN,Ocean or Sea,NaN,1.733989,0.050104,1.375259,5,coast,0,1


In [82]:
#countries with all coast cities
pool[pool.all_coast==1].FIPS_CNTRY.unique()

array(['WF', 'WS', 'AQ', 'FP', 'NE', 'TN', 'CW', 'PC', 'FK', 'SH', 'CJ',
       'BD', 'AA', 'NT', 'TK', 'BF', 'MT', 'PS', 'RP', 'SN', 'BX', 'RM',
       'FM', 'KR', 'CQ', 'GQ', 'CK', 'KT', 'TT', 'NF', 'TD', 'GY', 'VI',
       'RQ', 'VQ', 'AV', 'SC', 'AC', 'MH', 'ST', 'GP', 'DO', 'MB', 'VC',
       'BB', 'GJ', 'FG', 'SB', 'GK', 'JE', 'FO', 'IC', 'IM', 'CV', 'GI',
       'TP', 'MN', 'SV', 'EN', 'BA', 'AE', 'QA', 'MU', 'CN', 'MF', 'RE',
       'SE', 'MP', 'TV', 'BP', 'NH', 'FJ', 'NR'], dtype=object)

In [83]:
#countries with all coast cities
len(pool[pool.all_coast==1].FIPS_CNTRY.unique())

73

In [84]:
#countries with all inland cities
pool[pool.all_inland==1].FIPS_CNTRY.unique()

array(['PA', 'BL', 'CD', 'NG', 'CT', 'CG', 'SM', 'BK', 'RB', 'MJ', 'MK',
       'BU', 'RO', 'GG', 'UP', 'AM', 'IZ', 'JO', 'UG', 'OD', 'ER', 'ET',
       'SZ', 'LS', 'AU', 'EZ', 'HU', 'SI', 'KZ', 'MG', 'AF', 'KG', 'UZ',
       'TI', 'NP', 'BT', 'LA', 'CB', 'ML', 'UV', 'AN', 'BE', 'LU', 'LO',
       'BO', 'LH', 'MD', 'LG', 'AJ', 'TX', 'ZI', 'BC', 'ZA', 'BY', 'RW',
       'MI', 'WZ', 'LT', 'NC'], dtype=object)

In [85]:
#countries with all inland cities
len(pool[pool.all_inland==1].FIPS_CNTRY.unique())

59

In [86]:
73+59

132

In [87]:
pool['single_city_cntry'] = pool.groupby('FIPS_CNTRY')['FIPS_CNTRY'].transform(
    lambda x: 1 if len(x) == 1 else 0
)

/var/folders/1v/222nq_l91hg0lcmf2d4rfps80000gn/T/ipykernel_29684/1623177511.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pool['single_city_cntry'] = pool.groupby('FIPS_CNTRY')['FIPS_CNTRY'].transform(


In [88]:
pool

,FID,GMI_ADMIN,ADMIN_NAME,FIPS_CNTRY,CNTRY_NAME,STATUS,POP,POP_RANK,POP_CLASS,POP_SOURCE,...,TYPE__sea,ISO_CC__sea,SHAPE_Length__sea,SHAPE_Area__sea,distance_sea,POP_rank_updated,city_type,all_inland,all_coast,single_city_cntry
4,5,PRY-AMM,Amambay,PA,Paraguay,Provincial capital,0,0,,,...,Ocean or Sea,NaN,1.064543,0.007229,852.927751,5,inland,1,0,0
5,6,PRY-CNN,Canindeyu,PA,Paraguay,Provincial capital,0,0,,,...,Ocean or Sea,NaN,1.064543,0.007229,637.647981,7,inland,1,0,0
6,7,PRY-APA,Alto Parana,PA,Paraguay,Provincial capital,0,0,,,...,Ocean or Sea,NaN,0.355920,0.000982,650.833576,4,inland,1,0,0
7,8,PRY-ITA,Itapua,PA,Paraguay,Provincial capital,0,0,,,...,Ocean or Sea,NaN,0.591384,0.019002,716.754502,5,inland,1,0,0
14,15,BOL-LPA,La Paz,BL,Bolivia,National and provincial capital,2143000,2,"1,000,000 to 4,999,999",UN_Habitat_2020,...,Ocean or Sea,NaN,0.969157,0.051438,329.256259,2,inland,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2523,2524,SLB,Solomon Is.,BP,Solomon Is.,National capital,0,0,,,...,Ocean or Sea,NaN,0.506198,0.011142,0.583022,4,coast,0,1,1
2524,2525,VUT,Vanuatu,NH,Vanuatu,National capital,20094,7,"Less than 50,000",UN_Data_2010_2020,...,Ocean or Sea,NaN,0.043550,0.000072,0.536871,4,coast,0,1,1
2525,2526,FJI,Fiji,FJ,Fiji,National capital,0,0,,,...,Ocean or Sea,NaN,1.733989,0.050104,1.375259,5,coast,0,1,1
2526,2527,NCL,New Caledonia,NC,New Caledonia,National and provincial capital,99926,6,"50,000 to 99,999",UN_Data_2010_2020,...,Ocean or Sea,NaN,0.720061,0.008137,13.069313,6,inland,1,0,1


In [89]:
#pool.to_csv("processed/second_batch_pool20241118.csv", index=False) #EPSG:3857

In [91]:
#non-single-city countries with all inland cities
pool[(pool.all_inland==1)&(pool.single_city_cntry==0)].FIPS_CNTRY.unique()

array(['PA', 'BL', 'CD', 'NG', 'CT', 'CG', 'BU', 'RO', 'GG', 'UP', 'IZ',
       'JO', 'UG', 'OD', 'ET', 'SZ', 'AU', 'EZ', 'HU', 'KZ', 'MG', 'AF',
       'KG', 'UZ', 'TI', 'NP', 'BT', 'LA', 'CB', 'ML', 'UV', 'BE', 'LU',
       'LO', 'BO', 'AJ', 'TX', 'ZI', 'BC', 'ZA', 'BY', 'RW', 'MI', 'WZ',
       'LT'], dtype=object)

In [92]:
len(pool[(pool.all_inland==1)&(pool.single_city_cntry==0)].FIPS_CNTRY.unique())

45

In [96]:
#non-single-city countries with all inland cities
pool[(pool.all_inland==1)&(pool.single_city_cntry==0)].CNTRY_NAME.unique()

array(['Paraguay', 'Bolivia', 'Chad', 'Niger', 'Central African Republic',
       'Congo, DRC', 'Bulgaria', 'Romania', 'Georgia', 'Ukraine', 'Iraq',
       'Jordan', 'Uganda', 'South Sudan', 'Ethiopia', 'Switzerland',
       'Austria', 'Czech Republic', 'Hungary', 'Kazakhstan', 'Mongolia',
       'Afghanistan', 'Kyrgyzstan', 'Uzbekistan', 'Tajikistan', 'Nepal',
       'Bhutan', 'Laos', 'Cambodia', 'Mali', 'Burkina Faso', 'Belgium',
       'Luxembourg', 'Slovakia', 'Belarus', 'Azerbaijan', 'Turkmenistan',
       'Zimbabwe', 'Botswana', 'Zambia', 'Burundi', 'Rwanda', 'Malawi',
       'Eswatini', 'Lesotho'], dtype=object)

In [93]:
#non-single-city countries with all coast cities
pool[(pool.all_coast==1)&(pool.single_city_cntry==0)].FIPS_CNTRY.unique()

array(['RP', 'RQ', 'ST', 'TP', 'BA', 'AE', 'MU'], dtype=object)

In [95]:
#non-single-city countries with all coast cities
pool[(pool.all_coast==1)&(pool.single_city_cntry==0)].CNTRY_NAME.unique()

array(['Philippines', 'Puerto Rico', 'St. Lucia', 'Sao Tome & Principe',
       'Bahrain', 'United Arab Emirates', 'Oman'], dtype=object)

In [97]:
#single-city countries with inland capital
pool[(pool.all_inland==1)&(pool.single_city_cntry==1)].CNTRY_NAME.unique()

array(['San Marino', 'Bosnia & Herzegovina', 'Serbia', 'Montenegro',
       'North Macedonia', 'Armenia', 'Eritrea', 'Liechtenstein',
       'Slovenia', 'Andorra', 'Lithuania', 'Moldova', 'Latvia',
       'New Caledonia'], dtype=object)

In [98]:
#single-city countries with coast capital
pool[(pool.all_coast==1)&(pool.single_city_cntry==1)].CNTRY_NAME.unique()

array(['Wallis & Futuna', 'Samoa', 'American Samoa', 'French Polynesia',
       'Niue', 'Tonga', 'Cook Is.', 'Pitcairn Is.', 'Falkland Is.',
       'St. Helena', 'Cayman Is.', 'Bermuda', 'Aruba',
       'Netherlands Antilles', 'Turks & Caicos Is.', 'The Bahamas',
       'Malta', 'Palau', 'Singapore', 'Brunei', 'Marshall Is.',
       'Micronesia', 'Kiribati', 'Northern Mariana Is.', 'Guam',
       'Cocos Is.', 'Christmas I.', 'Timor Leste', 'Norfolk I.',
       'Trinidad & Tobago', 'Guyana', 'British Virgin Is.', 'Virgin Is.',
       'Anguilla', 'St. Kitts & Nevis', 'Antigua & Barbuda', 'Montserrat',
       'Guadeloupe', 'Dominica', 'Martinique',
       'St. Vincent & the Grenadines', 'Barbados', 'Grenada',
       'French Guiana', 'St. Pierre & Miquelon', 'Guernsey', 'Jersey',
       'Faroe Is.', 'Iceland', 'Isle of Man', 'Cape Verde', 'Gibraltar',
       'Monaco', 'Svalbard', 'Estonia', 'Qatar', 'Comoros', 'Mayotte',
       'Reunion', 'Seychelles', 'Mauritius', 'Tuvalu', 'Solomon Is.',